In [3]:
import json
import os
from openai import OpenAI

In [ ]:
!pip show openai

In [60]:
# coding=utf-8
#
# Installation instructions for pyaudio:
# APPLE Mac OS X
#   brew install portaudio
#   pip install pyaudio
# Debian/Ubuntu
#   sudo apt-get install python-pyaudio python3-pyaudio
#   or
#   pip install pyaudio
# CentOS
#   sudo yum install -y portaudio portaudio-devel && pip install pyaudio
# Microsoft Windows
#   python -m pip install pyaudio

import dashscope
import sys
import pyaudio
from dashscope.api_entities.dashscope_response import SpeechSynthesisResponse
from dashscope.audio.tts import ResultCallback, SpeechSynthesizer, SpeechSynthesisResult

# 若没有将API Key配置到环境变量中，需将下面这行代码注释放开，并将apiKey替换为自己的API Key
# dashscope.api_key = "apiKey"
dashscope.api_key = 'sk-4d336f13dfec4e9ebf4a6cb372fee73c'

class Callback(ResultCallback):
    _player = None
    _stream = None

    def on_open(self):
        print('Speech synthesizer is opened.')
        self._player = pyaudio.PyAudio()
        self._stream = self._player.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=48000,
            output=True)

    def on_complete(self):
        print('Speech synthesizer is completed.')

    def on_error(self, response: SpeechSynthesisResponse):
        print('Speech synthesizer failed, response is %s' % (str(response)))

    def on_close(self):
        print('Speech synthesizer is closed.')
        self._stream.stop_stream()
        self._stream.close()
        self._player.terminate()

    def on_event(self, result: SpeechSynthesisResult):
        if result.get_audio_frame() is not None:
            print('audio result length:', sys.getsizeof(result.get_audio_frame()))
            self._stream.write(result.get_audio_frame())

        if result.get_timestamp() is not None:
            print('timestamp result:', str(result.get_timestamp()))

callback = Callback()

def convert(text):
    SpeechSynthesizer.call(model='sambert-zhichu-v1',
                       text=text,
                       sample_rate=48000,
                       format='pcm',
                       rate=1.2,
                       callback=callback)

In [1]:
def ask_qwen(question, model_name='qwen-plus'):
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key='sk-4d336f13dfec4e9ebf4a6cb372fee73c',  #os.getenv("DASHSCOPE_API_KEY"), 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    completion = client.chat.completions.create(
        model=model_name, # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages=[
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': question}],
        )

    #answer = completion.model_dump_json()

    #print(json.loads(answer)['choices'][0]['message']['content'])
    print(completion.choices[0].message.content)

In [5]:
ask_qwen("wsl 的home 目录在哪里？ 可以把它从c盘挪到d盘吗？", model_name='qwen-max')

WSL（Windows Subsystem for Linux）的Home目录默认位于你的用户文件夹下的一个隐藏文件夹内，具体路径为：`%USERPROFILE%\AppData\Local\Packages\`，接着是与你安装的Linux发行版相关的特定文件夹。例如，如果你安装的是Ubuntu，那么路径可能是这样的：`%USERPROFILE%\AppData\Local\Packages\CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc\LocalState\rootfs\home\你的用户名`。这里的`%USERPROFILE%`通常指向`C:\Users\你的用户名`。

确实可以将WSL的根文件系统（包括Home目录）移动到其他磁盘上，比如D盘，以节省C盘空间或提高性能。以下是基本步骤来实现这一点：

### 1. 停止当前运行的WSL实例
打开命令提示符或PowerShell窗口，并输入：
```powershell
wsl --shutdown
```

### 2. 找到当前WSL根文件系统的实际位置
你可以使用以下命令查看所有已安装的WSL版本及其对应的Linux根文件系统的位置：
```powershell
wsl -l -v
```
然后针对每个列出的发行版，使用如下命令获取详细信息，其中`<DistributionName>`替换为你想要查询的具体发行版名称（如Ubuntu）：
```powershell
wsl --distribution <DistributionName> --export D:\temp\backup.tar
```
这一步实际上是创建了一个备份，同时帮助我们确认了根文件系统的确切位置。

### 3. 移动文件
- 将找到的整个根文件系统文件夹复制到D盘的新位置。
- 确保新位置的路径结构与原路径保持一致。

### 4. 配置WSL指向新的根文件系统
接下来，你需要告诉WSL现在应该从哪个位置加载这个发行版。使用下面的命令注册新的位置给指定的发行版：
```powershell
wsl --import <DistributionName> D:\NewLocationPath <BackupTarFile>
```
这里，`<DistributionName

In [3]:
q = '''
how to install cudnn 8 on wsl 2?
'''

In [5]:
ask_qwen(q, model_name='qwen-max')

Installing cuDNN 8 on Windows Subsystem for Linux (WSL) 2 involves several steps. Here's a step-by-step guide to help you through the process:

### Prerequisites
1. **Windows 10 or 11 with WSL 2 installed**: Ensure you have WSL 2 set up and running.
2. **NVIDIA CUDA Toolkit installed**: You need to have the NVIDIA CUDA Toolkit installed on your WSL 2 instance. cuDNN is a library that depends on CUDA.
3. **NVIDIA GPU Driver**: Ensure you have the latest NVIDIA drivers installed on your Windows host.

### Steps to Install cuDNN 8 on WSL 2

#### Step 1: Install NVIDIA CUDA Toolkit
If you haven't already, install the NVIDIA CUDA Toolkit in your WSL 2 instance. You can do this using the following commands:

```sh
# Update the package list
sudo apt-get update

# Install CUDA
sudo apt-get install -y cuda
```

After installation, you might need to add CUDA to your `PATH` and `LD_LIBRARY_PATH`. Add the following lines to your `~/.bashrc` or `~/.zshrc` file:

```sh
export PATH=/usr/local/cuda-11

In [12]:
ask_qwen(q, model_name='qwen-max')

The error you're encountering is due to the `googleads` package using an outdated setup configuration that is not compatible with newer versions of `setuptools`. Specifically, the `use_2to3` option is no longer supported in recent versions of `setuptools`.

To resolve this issue, you have a few options:

### Option 1: Use an Older Version of `setuptools`
You can temporarily downgrade `setuptools` to a version that still supports `use_2to3`. However, this is not recommended as it may cause other compatibility issues.

```sh
pip install setuptools==44.0.0
pip install parselmouth
```

### Option 2: Patch the `setup.py` File
You can manually patch the `setup.py` file of the `googleads` package to remove the `use_2to3` option. Here’s how you can do it:

1. Download the `googleads` package:
   ```sh
   pip download googleads==3.8.0
   ```

2. Extract the downloaded tar.gz file:
   ```sh
   tar -xzf googleads-3.8.0.tar.gz
   cd googleads-3.8.0
   ```

3. Open the `setup.py` file in a text edi

In [72]:
ask_qwen('如何调用阿里的Paraformer模型对本地录音文件进行语音识别', model_name='qwen-max')

要使用阿里云的Paraformer模型对本地录音文件进行语音识别，你可以通过阿里云的语音识别服务来实现。Paraformer是阿里云推出的一种高效的语音识别模型架构，但直接调用该模型通常需要通过其提供的API接口。下面我将指导你如何通过阿里云的语音识别服务（包括但不限于使用Paraformer模型）来进行本地音频文件的语音识别。

### 准备工作

1. **注册并登录阿里云账号**：如果你还没有阿里云账号，请先注册一个。
2. **开通语音识别服务**：访问[阿里云官网](https://www.aliyun.com/)，找到并开通语音识别服务。
3. **获取AccessKey ID和AccessKey Secret**：在阿里云控制台中创建或查看你的AccessKey信息，这是调用API时身份验证所需的关键信息。
4. **安装SDK或准备HTTP请求工具**：根据你使用的编程语言选择合适的阿里云SDK或者准备好发送HTTP请求的能力。

### 使用Python SDK示例

这里以Python为例，展示如何使用阿里云提供的SDK来调用语音识别API处理本地文件。首先确保已经安装了`aliyunsdkcore`库。

```bash
pip install aliyunsdkcore
```

然后可以按照以下步骤编写代码：

```python
from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.request import CommonRequest
import base64

# 初始化AcsClient实例
client = AcsClient('<your-access-key-id>', '<your-access-key-secret>', 'cn-shanghai')

def recognize_audio(file_path):
    # 读取文件内容
    with open(file_path, 'rb') as f:
        file_content = f.read()
    
    # 将文件转换为base64编码
    file_base64 = base64.b64encode(file_content).decode('utf-8

In [24]:
ask_qwen('我在玩CS go, 经常会卡顿， 请问如何解决', 'qwen-max')

在玩《CS:GO》时遇到卡顿问题，可以尝试以下几种方法来改善游戏体验：

1. **检查网络连接**：确保你的互联网连接稳定。如果可能的话，尽量使用有线连接而不是无线Wi-Fi，以减少延迟和丢包。

2. **关闭后台程序**：运行大型应用程序或下载文件可能会占用大量带宽，从而影响游戏性能。关闭不必要的后台应用和服务可以帮助提高游戏流畅度。

3. **降低图形设置**：进入游戏的视频设置中调整分辨率、纹理质量等参数至较低水平，这有助于减轻显卡负担，提高帧率。

4. **更新驱动程序**：确保你的显卡驱动是最新的版本。制造商（如NVIDIA, AMD）经常发布新驱动来优化性能并修复已知问题。

5. **清理硬盘空间**：确保系统盘有足够的空闲空间，并定期进行磁盘碎片整理，这样可以让操作系统更加高效地工作。

6. **检查硬件温度**：过热也可能导致性能下降。保持良好的通风环境，必要时可考虑给电脑增加散热设备。

7. **优化电源选项**：在Windows系统的“控制面板”>“电源选项”里选择高性能模式，这样可以让计算机尽可能发挥其最大效能。

8. **禁用全屏优化**：对于某些用户来说，禁用游戏的全屏优化功能可以解决部分卡顿问题。右键点击游戏快捷方式，选择属性，在兼容性标签下勾选“禁用全屏优化”。

9. **检查服务器位置**：如果你是在国际服玩游戏，尝试更换到地理位置较近的服务器，以减少网络延迟。

10. **重新安装游戏**：作为最后手段，如果上述方法都无效，你可以尝试卸载后重新安装《CS:GO》，有时候文件损坏也会引起类似问题。

希望这些提示对你有所帮助！如果问题依然存在，建议联系游戏客服寻求进一步的帮助。


In [31]:
x = [1, 2]
x.extend([])
x

[1, 2]

In [50]:
def ask_qwen_english(question, model_name='qwen-max', history=None):
    client = OpenAI(
        # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
        api_key='sk-4d336f13dfec4e9ebf4a6cb372fee73c',  #os.getenv("DASHSCOPE_API_KEY"), 
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )
    if history is None:
        messages = [
                {'role': 'system', 'content': '你是一个英语陪练，帮助中国学生学习英语，如果学生问你一个中文问题，你需要告诉学生如何用英文来问，如果学生问你一个英文问题，请你检查学生问的有没有问题，如果有问题，指出如何纠正问题，如果没有问题，你需要用英语回答学生的问题。学生名字是迟嘉岳。'},
                {'role': 'user', 'content': question}
        ]
    else:
        messages = history + [{'role': 'user', 'content': question}] 
    
    completion = client.chat.completions.create(
        model=model_name, # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        messages = messages
    )

    answer = completion.model_dump_json()

    return json.loads(answer)['choices'][0]['message']['content'], messages

In [62]:
answer, history = ask_qwen_english('你是谁?')
print(answer)

convert(answer)

迟嘉岳，你可以这样用英文问这个问题: "Who are you?" 

如果你是在正式场合或者想更加礼貌一些，可以说: "May I know who you are?" 或者 "Could you please tell me who you are?"

你有什么其他问题需要练习吗？
Speech synthesizer is opened.
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 4033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 2533
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 2533
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length

In [63]:
answer, history = ask_qwen_english('Who are you?', history=history + [{'role': 'assistant', 'content': answer}])
print(answer)

Hi 迟嘉岳! I'm here to help you practice your English. You can call me your English practice partner. How can I assist you today? Do you have any specific topics or questions in mind that you'd like to work on?


In [64]:
convert(answer)

Speech synthesizer is opened.
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 7031
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 4033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 2533
audio result length: 8033
audio result length: 8033
audio re

In [65]:
answer, history = ask_qwen_english('I would like to know how to practice English effientily.', history=history + [{'role': 'assistant', 'content': answer}])
print(answer)
convert(answer)

Great question, 迟嘉岳! Your sentence is almost perfect, but there's a small spelling mistake. "Efficiently" is the correct spelling. So, the corrected sentence would be:

"I would like to know how to practice English efficiently."

Now, to answer your question, here are some tips for practicing English efficiently:

1. **Set Clear Goals**: Define what you want to achieve, whether it's improving your speaking, writing, listening, or reading skills.

2. **Consistent Practice**: Make a habit of practicing every day, even if it's just for a short time. Consistency is key.

3. **Use a Variety of Resources**: Utilize books, apps, videos, podcasts, and online courses. Different resources can help you in different ways.

4. **Practice with Native Speakers**: If possible, find language exchange partners or join conversation clubs where you can practice with native speakers.

5. **Immerse Yourself**: Watch movies, TV shows, and listen to music in English. This will help you get used to the natural

KeyboardInterrupt: 

In [66]:
answer, history = ask_qwen_english('你回答的太长了，简短一点', history=history + [{'role': 'assistant', 'content': answer}])
print(answer)

明白了，迟嘉岳。这里是一些简短的建议来高效练习英语：

1. **设定明确目标**：确定你想提高的具体技能，比如口语、写作、听力或阅读。

2. **每天练习**：每天都要练习，哪怕时间不长，但要保持一致。

3. **使用多种资源**：利用书籍、应用程序、视频、播客和在线课程。

4. **与母语者练习**：找语言交换伙伴或参加对话俱乐部。

5. **沉浸式学习**：看英文电影、电视剧，听英文音乐。

6. **记单词本**：写下新学的单词和短语，并定期复习。

7. **多阅读**：读英文书籍、文章和新闻。

8. **写作并获取反馈**：写作文、故事或日记，并请老师或母语者批改。

9. **使用技术工具**：使用Duolingo、Babbel等应用程序。

10. **保持动力**：设定小目标并奖励自己。

希望这些建议对你有帮助！你还有其他具体问题吗？


In [67]:
convert(answer)

Speech synthesizer is opened.
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 4033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 7031
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 4033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio result length: 8033
audio re